# Imports

In [2]:
import polars as pl
import numpy as np
import folium
import utils

# Read Data

In [3]:
mines_csv = pl.read_csv(
    r"C:\Users\16617\Desktop\Code_Library\DMR Analysis\DMR_All_Mines.csv",
    ignore_errors=True,
)

# Clean and Filter Data

In [70]:
mines_csv = mines_csv[utils.get_cols_to_keep()]

active_open_pit_mines = mines_csv.filter(
    (pl.col("MineStatus").str.contains("ACTIVE"))
    & (pl.col("Opt_Type").str.contains("OPEN PIT"))
    & (~pl.col("PriProduct").str.contains("NOT REPORT"))
)

holloway = active_open_pit_mines.filter(pl.col("Operator").str.contains("HOLLOWAY"))

labels, index = np.unique(
    active_open_pit_mines.select(pl.col("PriProduct")).to_numpy().squeeze(),
    return_inverse=True,
)

colors = [
    'red',
    'blue',
    'black',
    'darkgreen',
    'darkblue',
    'darkpurple',
    'red',
    'blue',
    'black',
    'darkgreen',
    'darkblue',
    'darkpurple',
    'red',
    'blue',
    'black',
    'darkgreen',
    'darkblue',
    'darkpurple',
    'red',
    'blue',
    'black',
    'darkgreen',
    'darkblue',
    'darkpurple',
    'red',
]

color_dict = {labels[i]: color for i, color in enumerate(colors)}
active_open_pit_mines = active_open_pit_mines.with_columns(pl.col("PriProduct").map_dict(color_dict).alias("color"))


# Plot Data

In [76]:
m = folium.Map([35, -119], zoom_start=8)
for name, mine in active_open_pit_mines.groupby("Mine_ID"):
    folium.CircleMarker(
        location = [mine["Latitude"].to_list()[0],mine["Longitude"].to_list()[0]],
        radius=5,
        color=mine["color"].to_list()[0],
        fill_color = mine["color"].to_list()[0],
        tooltip=mine["PriProduct"].to_list()[0],
        popup=mine["Operator"].to_list()[0],
    ).add_to(m)

folium.Marker(
        location = [holloway["Latitude"].to_list()[0],holloway["Longitude"].to_list()[0]],
        tooltip=holloway["PriProduct"].to_list()[0],
        popup=holloway["Operator"].to_list()[0],
        icon=folium.Icon(color="green")
    ).add_to(m)

m.save(f'dmr_map.html')